## 1. Preprocessing

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
### 
reader = open("similarity_dataset.txt").readlines()
for line in range(1, len(reader)):
     line_list = reader[line].split("\t")[0:3]
     filtered_gold_standard.update({(line_list[0], line_list[1]): float(line_list[2])})

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma

from collections import Counter
norm_doc = tuple(Counter(lemmatize(word.lower()) for token in raw_doc for word in token if word.isalpha()) 
                                                   for raw_doc in brown.paras())
dfs = Counter()
for processed_doc in norm_doc:
    for term in processed_doc.keys():
        dfs[term] += 1

filtered_gold_standard = {key: value 
                      for key, value in filtered_gold_standard.items() if dfs[key[0]] > 7 and dfs[key[1]] > 7}
###
# Your answer ENDS HERE
###



print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('tennis', 'racket'): 7.56, ('law', 'lawyer'): 8.38, ('movie', 'star'): 7.38, ('movie', 'critic'): 6.73, ('movie', 'theater'): 7.92, ('space', 'chemistry'): 4.88, ('alcohol', 'chemistry'): 

<b>For your testing: </b>

In [2]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [3]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

In [4]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###
final_gold_standard = {}
for key, value in filtered_gold_standard.items():
    passing = True
    for word in key:
        synsets = wordnet.synsets(word)
        if synsets[0].pos() is 'n' and (len(synsets) is 1 or max(synsets[0].lemmas(),key=lambda x:x.count()).count() > 4 * max(synsets[1].lemmas(),key=lambda x:x.count()).count()):
            continue
        passing = False
        break
    if passing:
        final_gold_standard[key] = value
###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

24
{('train', 'car'): 6.31, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'phone'): 1.62, ('money', 'cash'): 9.08, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'fear'): 6.85, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'discipline'): 5.58, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [5]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [6]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

In [7]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###
brown_ic = wordnet_ic.ic('ic-brown.dat')
for word1, word2 in final_gold_standard:
    lin_similarities[(word1, word2)] = wordnet.synsets(word1)[0].lin_similarity(wordnet.synsets(word2)[0], brown_ic)
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


{('train', 'car'): 0.6786666052669672, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'phone'): -0.0, ('money', 'cash'): 0.7888839126424345, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'fear'): 0.06578656336760624, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'discipline'): 0.7275617910923693, ('planet', 'moon'): 0.7808882364067532, ('planet', 'galaxy'): -0.0}


<b>For your testing:</b>

In [8]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NMPI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

In [9]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
from math import log2
number = sum(len(doc) for doc in norm_doc)
for x, y in final_gold_standard:
    pxy, px, py = 0, 0, 0
    for doc in norm_doc:
        if x in doc:
            px += 1
        if y in doc:
            py += 1
        if x in doc and y in doc:
            pxy += 1
    px /= number
    py /= number
    if pxy:
        NPMI_similarities[(x, y)] = log2(px * py) / log2(pxy / number) - 1
    else:
        NPMI_similarities[(x, y)] = - 1
###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('train', 'car'): 0.2748859551437035, ('professor', 'doctor'): -1, ('student', 'professor'): 0.535495995400926, ('stock', 'phone'): -1, ('money', 'cash'): 0.44493834723517245, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7161994042283006, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6246376972254175, ('baby', 'mother'): 0.5149353890388502, ('car', 'automobile'): 0.5430334549802616, ('coast', 'shore'): 0.5861510629843374, ('brother', 'monk'): 0.42993185256931743, ('coast', 'hill'): 0.33931028834846244, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892886, ('psychology', 'fear'): -1, ('psychology', 'doctor'): 0.4613281583726412, ('psychology', 'mind'): 0.4461666043435182, ('psychology', 'health'): -1, ('psychology', 'discipline'): -1, ('planet', 'moon'): 0.658000616232332, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [10]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

In [11]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
vocabulary = tuple(set(word for doc in norm_doc for word in doc.keys()))

from numpy import array
norm_doc = array(tuple(array(tuple(0. if word not in doc else doc[word] for doc in norm_doc), 'float64') 
                                     for word in vocabulary), 'float64')
 
from sklearn.decomposition import TruncatedSVD
norm_doc = TruncatedSVD(n_components=500).fit_transform(norm_doc)

from scipy.spatial.distance import cosine
for pair in final_gold_standard:
    LSA_similarities[pair] = 1 - cosine(norm_doc[vocabulary.index(pair[0])], norm_doc[vocabulary.index(pair[1])])
###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('train', 'car'): 0.0009416027757649381, ('professor', 'doctor'): 0.03688406170163694, ('student', 'professor'): 0.10427950027244204, ('stock', 'phone'): -0.01862100311489634, ('money', 'cash'): 0.125626052051793, ('bishop', 'rabbi'): 0.05460526894072226, ('football', 'basketball'): 0.2089971318150372, ('football', 'tennis'): 0.09295331731850986, ('alcohol', 'chemistry'): 0.24664055826245213, ('baby', 'mother'): 0.22182607044915925, ('car', 'automobile'): 0.25922375699806854, ('coast', 'shore'): 0.2738911409456255, ('brother', 'monk'): 0.03209572212985501, ('coast', 'hill'): 0.1784991585523964, ('forest', 'graveyard'): 0.014399391869365896, ('monk', 'slave'): 0.00724520296512543, ('coast', 'forest'): 0.03398415311525138, ('psychology', 'fear'): 0.01943396678891507, ('psychology', 'doctor'): 0.03780627330417763, ('psychology', 'mind'): 0.16558659150861277, ('psychology', 'health'): -0.033752544392562145, ('psychology', 'discipline'): 0.11152197816044718, ('planet', 'moon'): 0.508958466

<b>For your testing:</b>

In [12]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

In [13]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###
pearson_correlations['lin' ] = pearsonr(list(lin_similarities.values() ), list(final_gold_standard.values()))[0]
pearson_correlations['NPMI'] = pearsonr(list(NPMI_similarities.values()), list(final_gold_standard.values()))[0]
pearson_correlations['LSA' ] = pearsonr(list(LSA_similarities.values() ), list(final_gold_standard.values()))[0]
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5233821518184203, 'NPMI': 0.38863324558393886, 'LSA': 0.4776193266054849}


<b>For your testing:</b>

In [14]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)